In this notebook, we'll use the Blankly package to experiment with a bot based on the Kelly Criterion. The Kelly Criterion deals with position sizing -- given how confident we are in our model's prediction for an asset's behavior, the Kelly Criterion outputs the optimal size for the position.

In [ ]:
!pip install blankly #install blankly

We'll initialize the basics of our Blankly environment with the command '*blankly init*'. Once done, we get template .json files that we'll need for configuring backtests. Most importantly, we'll need to input our API keys into keys.json. 

In [ ]:
!blankly init

Here are our imports -- we just need Blankly for this strategy.

In [4]:
import blankly

This section is where we'll initialize the framework for our Kelly price event. We'll create an interface to interact with the exchange as well as set a resolution and download historical data. We'll also estimate our win/loss "probabilities" using a simple RSI bucket strategy. We'll split price movement data into bands of size 10 (RSI 40 to RSI 50 etc.). Then, we'll use the historically observed data to calculate the probability of an increase as well as a win/loss ratio, which we then use to calculate the optimal Kelly size.

In [76]:
def init_kelly(symbol, state: blankly.StrategyState):
    interface = state.interface
    resolution = state.resolution
    variables = state.variables

    #Get price data
    variables['history'] = interface.history(symbol, 300, resolution, return_as='list')['close']
    rsi = blankly.indicators.rsi(variables['history'])
    '''
    Create RSI buckets, each of which corresponds to a size-10 range of RSI values. 
    '''
    buckets = [[0,0,0,0] for i in range(10)]
    '''For each datapoint, count whether price increases/decreases, size of increase/decrease,
     and put it in appropriate bucket'''
    for i in range(len(variables['history']) - 15):
      r = rsi[i]
      p = variables['history'][i + 15]
      cp = variables['history'][i + 14]
      ind = int(r//10)
      if r < 0:
        ind = 0
      elif r > 90:
        ind = 9
      if cp < p:
        buckets[ind][0]+=1
        buckets[ind][1]+=1
        buckets[ind][3]+=((p - cp)/cp)
      elif cp > p:
        buckets[ind][0]+=0
        buckets[ind][1]+=1
        buckets[ind][2]+=((p - cp)/cp)
    ratios = []
    '''Calculates win/loss ratios'''
    for elem in buckets:
      if elem[0]==0:
        ratios.append(0)
      elif elem[1] - elem[0] == 0:
        ratios.append(1)
      else:
        ratios.append((-elem[3]/elem[0])/(elem[2]/(elem[1] - elem[0])))
    '''Calculates win/loss probabilities'''
    probs = [(elem[0]/elem[1]) if elem[1]!=0 else 0 for elem in buckets]
    '''Calculates Kelly sizing according to formula
    W - (1-W)/R
    '''
    variables['kelly_sizes'] = [max(0,probs[i] - (1-probs[i])/ratios[i]) if ratios[i]!=0 else 0 for i in range(len(probs))]
    print(variables['kelly_sizes'])
    state.variables['owns_position'] = False
    #print(len(variables['history']))
    #print(len(rsi))

This section is where we implement the logic of the trading algorithm. After getting the newest price, we calculate the RSI, and based on the observed historical data, we choose how much to scale into the position.

In [75]:
def price_kelly(price,symbol,state: blankly.StrategyState):
    state.variables['history'].append(price) #Add latest price to current list of data
    '''Here, we pull the data from the last few days, prepare it,
    and run the necessary indicator functions to feed into our model
    '''
    rsi = blankly.indicators.rsi(state.variables['history'])
    '''Clear previous day's position'''
    curr_value = blankly.trunc(state.interface.account[state.base_asset].available, 2) #Amount of asset available
    if curr_value > 0:
      state.interface.market_order(symbol, side='sell', size=curr_value)
    '''Determine bucket based off RSI'''
    ind = int(rsi[-1]//10)
    ind = max(0,ind)
    ind = min(9, ind)
    buy = blankly.trunc(state.variables['kelly_sizes'][ind] * state.interface.cash/price, 2) #Buy appropriate amount
    if buy > 0:
      state.interface.market_order(symbol, side='buy', size=buy)

Baseline strategy -- 30/70 RSI Bot. We might not get returns as large, as our typical buy sizes are much smaller than in this model, but we hope to at least get better performance in terms of ratios.

In [55]:
def price_event(price, symbol, state: blankly.StrategyState):
    """ This function will give an updated price every 15 seconds from our definition below """
    state.variables['history'].append(price)
    rsi = blankly.indicators.rsi(state.variables['history'])
    if rsi[-1] < 30 and not state.variables['owns_position']:
        # Dollar cost average buy
        buy = blankly.trunc(state.interface.cash/price, 2)
        state.interface.market_order(symbol, side='buy', size=buy)
        state.variables['owns_position'] = True
    elif rsi[-1] > 70 and state.variables['owns_position']:
        # Dollar cost average sell
        curr_value = blankly.trunc(state.interface.account[state.base_asset].available, 2)
        state.interface.market_order(symbol, side='sell', size=curr_value)
        state.variables['owns_position'] = False

Here, we test our model! We start with $10,000 and connect to Alpaca's API through Blankly. After creating a Blankly strategy and adding our price event, we can run and see the results. We want to compare our strategy's performance to a baseline that just naively buys and sells as much as possible when RSI is less than 30 or above 70.

In [77]:
exchange = blankly.Alpaca() #Connect to Alpaca API
strategy = blankly.Strategy(exchange) #Initialize a Blankly strategy
strategy.add_price_event(price_event, symbol='SPY', resolution='1d', init=init_kelly) #Add our price event and initialization. Using the Kelly initialization is fine.
results = strategy.backtest(to='1y', initial_values={'USD': 10000}) #Backtest one year starting with $10,000
print(results)

INFO: No portfolio name to load specified, defaulting to the first in the file: (another cool portfolio). This is fine if there is only one portfolio in use.



Initializing...
[0, 0, 0, 0.7157529186475641, 0.30227335073055467, 0.18090017256614732, 0.008518120204588253, 0.7897199745088698, 0, 0]

Backtesting...
Progress: [##########] 100% Done...

Historical Dataframes: 
Account History: 
            USD    SPY          time  Account Value (USD)
0    10000.0000   0.00  1.617595e+09           10000.0000
1    10000.0000   0.00  1.617595e+09           10000.0000
2    10000.0000   0.00  1.617682e+09           10000.0000
3    10000.0000   0.00  1.617768e+09           10000.0000
4    10000.0000   0.00  1.617854e+09           10000.0000
..          ...    ...           ...                  ...
358      1.1886  23.07  1.648440e+09           10517.4174
359  10648.6857   0.00  1.648526e+09           10648.6857
360  10648.6857   0.00  1.648613e+09           10648.6857
361  10648.6857   0.00  1.648699e+09           10648.6857
362  10648.6857   0.00  1.648786e+09           10648.6857

[363 rows x 4 columns]
Account Returns: 
             time     value
0 

INFO: Make an account to take advantage of the platform backtest viewer: https://app.blankly.finance/5Z9MWfnUzwIyy9Qv385a/1Ss7zybwN8aMAbWb3lSH/aG3LE1LzHnY24oqtBMS3/backtest


When we run our baseline, we see a slight profit along with a Sharpe Ratio of 0.53 and Sortino of 0.39 -- not amazing in terms of reward/risk.

In [78]:
exchange = blankly.Alpaca() #Connect to Alpaca API
strategy = blankly.Strategy(exchange) #Initialize a Blankly strategy
strategy.add_price_event(price_kelly, symbol='SPY', resolution='1d', init=init_kelly) #Add our price event and initialization
results = strategy.backtest(to='1y', initial_values={'USD': 10000}) #Backtest one year starting with $10,000
print(results)

INFO: No portfolio name to load specified, defaulting to the first in the file: (another cool portfolio). This is fine if there is only one portfolio in use.



Initializing...
[0, 0, 0, 0.7157529186475641, 0.30227335073055467, 0.18090017256614732, 0.008518120204588253, 0.7897199745088698, 0, 0]

Backtesting...
Progress: [##########] 100% Done...

Historical Dataframes: 
Account History: 
            USD    SPY          time  Account Value (USD)
0    10000.0000   0.00  1.617595e+09         10000.000000
1     9918.7280   0.20  1.617595e+09         10000.000000
2     9918.7280   0.20  1.617682e+09          9999.952000
3     9918.7280   0.20  1.617768e+09         10000.046000
4     9918.7280   0.20  1.617854e+09         10000.432000
..          ...    ...           ...                  ...
358  10664.6571   0.21  1.648440e+09         10760.383493
359   2264.8111  18.41  1.648526e+09         10761.578393
360  10619.3770   0.20  1.648613e+09         10711.134993
361   8770.2471   4.29  1.648699e+09         10709.798993
362   8774.7763   4.28  1.648786e+09         10713.273893

[363 rows x 4 columns]
Account Returns: 
             time     value
0 

INFO: Make an account to take advantage of the platform backtest viewer: https://app.blankly.finance/5Z9MWfnUzwIyy9Qv385a/1Ss7zybwN8aMAbWb3lSH/aG3LE1LzHnY24oqtBMS3/backtest


In contrast, when we run our Kelly-based allocation model, we end up with a much better reward/risk. We had slightly better returns with significantly less capital deployed, resulting in a 0.97 Sharpe Ratio and 1.3 Sortino Ratio. This model isn't super amazing, as is expected when we look only at a single RSI datapoint to predict the next day's move, but the main point is that the Kelly-based RSI model outperformed the naive RSI model. With a more effective model, adding Kelly could result in significantly better performance.